In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels
from scipy import stats
from statsmodels.stats.multitest import multipletests
from statannot import add_stat_annotation

In [1]:
class LONGIT(object):
    """Class to deal with the EC50 excel file generated by the Lai lab
    """
    def __init__(self):
        self.infile = "nan"

    def loadfiletodf(self, infile):
        self.infile = infile
        self.df = pd.DataFrame()
        self.df = pd.read_excel(infile,index_col=None,header=0)
        
    @staticmethod
    def samples_with_missing_elements(df):
        indx_missing = pd.isnull(df).any(1).to_numpy().nonzero()[0]
        df_missing = pd.DataFrame()
        for item in indx_missing:
            df_missing = df_missing.append(df_1.loc[item], ignore_index=False)
        pt_missing_set = set(df_missing['Pt#'])
        df = df.dropna()
        return(df_missing, df)
    
    @staticmethod
    def reformat_onset(df):
        df3 = pd.DataFrame()
        df3 = df3.append(df, ignore_index=False)
        for index, row in df.iterrows():
            sampleID, onset = str(row['sampleID_onset']).split(sep='-')
            # print(sampleID, onset)
            df3.at[index, 'sampleID_onset'] = onset
        return(df3)
    
    @staticmethod
    def incoherent_ec50(df, thresh):
        df4 = pd.DataFrame()
        df4 = df4.append(df, ignore_index=False)
        df4_ec50dif = pd.DataFrame()
        for index, row in df.iterrows():
            eca, ecb = str(row['run1_EC50']), str(row['run2_EC50'])
            eca = eca.replace("~","")
            ecb = ecb.replace("~","")
            df4.at[index,'run1_EC50'] = eca
            df4.at[index,'run2_EC50'] = ecb
            if (float(eca)/float(ecb) >= thresh or float(ecb)/float(eca) >= thresh) and abs(float(eca)-float(ecb)) > 1:
                df4_ec50dif = df4_ec50dif.append(df.loc[index], ignore_index=False)
                df4 = df4.drop([index])
            else:
                av = (float(eca) + float(ecb)) /2
                df4.at[index, 'average_EC50'] = av
        return(df4_ec50dif, df4)
    
    @staticmethod
    def transform_df_for_plotting(df, indx_col, colname_col, value_col):
        df5 = pd.DataFrame()
        for index, row in df.iterrows():
            pt = row[indx_col]
            day = row[colname_col]
            ec50 = row[value_col]
#             print('joder', pt, day)
            df5.at[pt,int(day)] = ec50
        df5 = df5.reindex(sorted(df5.columns), axis=1)
        return(df5)
    
    @staticmethod
    def plot_points_per_day(df, outfile):
        day_ar = np.array(df.columns)
        count_ar = np.zeros((0))
        for (col_name,col_data) in df.iteritems():
            data_ar = col_data.to_numpy()
            data_clean=data_ar[np.logical_not(np.isnan(data_ar))]
            count_ar = np.append(count_ar, data_clean.size)
        # print(day_ar)
        # print(count_ar)
        sns.set(style="whitegrid", color_codes=True)
        pal = sns.color_palette("coolwarm",55)
        order = count_ar.argsort()
        rank = order.argsort()
        fig=plt.figure(figsize=(18, 10))
        sns.barplot(x=day_ar, y=count_ar, palette=np.array(pal[::-1])[rank])
        plt.xticks(rotation=90)
        plt.xlabel('Days after onset')
        plt.ylabel('Number of patients')
        plt.savefig(outfile, dpi=300)
    
    @staticmethod
    def plot_points_per_patient(df, outfile):
        count_ar = np.zeros((0))
        for indx,row in df.iterrows():
            data_ar = np.array(row)
            data_clean = data_ar[np.logical_not(np.isnan(data_ar))]
            count = data_clean.size
            count_ar = np.append(count_ar, count)

        fig=plt.figure(figsize=(18, 5))
        sns.set(style="whitegrid")
        g=sns.distplot(count_ar, kde=False, bins = 17, rug=False, hist_kws={"rwidth":1,'edgecolor':'black',\
                                                                                     'alpha':0.6,'align':'right'})
        # plt.yaxis.set_major_locator(ticker.MultipleLocator(0.005))
        plt.yticks(np.arange(0,30,2))
        plt.xticks(np.arange(0,25,1))
        plt.xlabel('Number of days')
        plt.ylabel('Number of patients')
        plt.savefig(outfile, dpi=300)
        
    @staticmethod
    def plot_clustermap_availdata(df, outfile):
        df6 = df.notna()
        df6 = df6*1
        cg = sns.clustermap(df6,col_cluster=False, figsize=(10,20), cmap='Blues_r',
                            vmax=1.3,cbar=False, linewidths=.1, yticklabels=True,
                            xticklabels=True)
        cg.cax.set_visible(False) #hide colorbar
        cg.ax_row_dendrogram.set_visible(False) #hide row dendrogram
        cg.ax_heatmap.set_yticklabels(cg.ax_heatmap.get_ymajorticklabels(), fontsize = 8)
        cg.ax_heatmap.set_xticklabels(cg.ax_heatmap.get_xmajorticklabels(), fontsize = 8)
        cg.savefig(outfile, dpi=300)

        
    @staticmethod
    def plot_ec50(indx_ar, df, outdir, outfile, targetVar):
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        col_ar = np.array(df.columns)
        f = plt.figure(figsize=[18,10])
        for indx in indx_ar: 
            val_ar = np.array(df.loc[indx])
            col_clean = col_ar[np.isfinite(val_ar)]
            val_clean = val_ar[np.isfinite(val_ar)]
            plt.plot(col_clean, val_clean,'o-', label=indx)
        f.legend(fontsize=5)
        plt.xticks(np.arange(0,col_ar.max()+2,2))
        plt.xlabel('Days after onset')
        plt.ylabel(targetVar)
        plt.title(outfile)
        f.savefig(outdir + '/' + outfile,dpi=300)
        f.clear()
        plt.close(f)
        
    @staticmethod
    def plot_log10ec50(indx_ar, df, outdir, outfile):
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        col_ar = np.array(df.columns)
        f = plt.figure(figsize=[18,10])
        for indx in indx_ar: 
            val_ar = np.array(df.loc[indx])
            col_clean = col_ar[np.isfinite(val_ar)]
            val_clean = val_ar[np.isfinite(val_ar)]
            val_transform = -np.log10(val_clean)
            plt.plot(col_clean, val_transform,'o-', label=indx)
        f.legend(fontsize=5)
        plt.xticks(np.arange(0,col_ar.max()+2,2))
        plt.ylim((0,6))
        plt.xlabel('Days after onset')
        plt.ylabel('-log10(EC50)')
        plt.title(outfile)
        f.savefig(outdir + '/' + outfile,dpi=300)
        f.clear()
        plt.close(f)
    
    @staticmethod
    def add_survival_to_df(df, survival_index_ar, nonsurvival_indx_ar):
        df7 = pd.DataFrame(columns=["Day","Var","Category"])
        for indx, row in df.iterrows():
            if not indx in survival_indx_ar and not indx in nonsurvival_indx_ar:
                print("skipping index", indx)
                continue
            for day, ec50 in row.items():
                cat = ""
                if indx in survival_indx_ar:
                    cat = "Survival"
                else:
                    cat = "Non-survival"
                new_row = pd.Series(data={'Day':day, 'Var':ec50, 'Category':cat}, name=indx)
                df7 = df7.append(new_row,ignore_index=False)
        return(df7)
    
    @staticmethod
    def add_category_to_df(df_ec50, df_cat):
        # this function needs to subsitute add_survival_to_df
        df_ec50_cat = pd.DataFrame(columns=["Day","Var","Category"])
        for indx, row in df_ec50.iterrows():
            if not indx in df_cat.index:
                print("skipping index", indx)
                continue
            cat = df_cat.at[int(indx),'Category']
            for day, ec50 in row.items():
                new_row = pd.Series(data={'Patient':indx, 'Day':day, 'Var':ec50, 'Category':cat})
                df_ec50_cat = df_ec50_cat.append(new_row,ignore_index=True)
        return(df_ec50_cat)

    @staticmethod
    def calculate_pval(day, df, min_size, convert2Log):
        df = df.loc[df['Day'] == day]
        df = df.dropna(how='any')
        if convert2Log == True:
            df['Var'] = -np.log10(df['Var'])
        cat_set = set(df['Category'].tolist())
        print(cat_set)
        if len(cat_set) != 2:
            return None
        
#         print('Day', day)
        #man-whitney test
        survival_ar = df.loc[df['Category'] == 'Survival']['Var'].tolist()
#         print(survival_ar)
        nonSurvival_ar = df.loc[df['Category'] == 'Non-survival']['Var'].tolist()
#         print(nonSurvival_ar)
#         print('length survival', len(survival_ar))
#         print('length nonsurvival', len(nonSurvival_ar))
        if len(survival_ar) < min_size or len(nonSurvival_ar) < min_size:
            return None
#         print('calculating pvalue...')
        ustat,pval = stats.mannwhitneyu(survival_ar, nonSurvival_ar, alternative='two-sided')
#         print('Pval',pval)
#         print('Ustat', ustat)
        return(pval)
    
    def multipletesting_day_to_pval(self, df, min_size, convert2Log):
        day_st = set(df['Day'].tolist())
        
        # Get uncorrected pvalues
        day_2_pval = {}
        for day in day_st:
            pval = self.calculate_pval(day, df, min_size, convert2Log)
            if isinstance(pval, float):
                day_2_pval[day] = pval

        # multiple testing correction      
        day_ls = list(day_2_pval.keys())
        pval_ls = list(day_2_pval.values())
#         print('Length of vector in multiple correction:', len(pval_ls))
        pvalcor_ls = []
        day_2_pvalcor = {}
        if len(pval_ls) > 0:
            pvalcor_ls = multipletests(pval_ls, alpha=0.05, method='fdr_bh',
                                       is_sorted=False, returnsorted=False)
    #         print(pval_ls)
    #         print(pvalcor_ls)
            day_2_pvalcor = {}
            for i,item in enumerate(pvalcor_ls[1]):
    #             print(pval_ls[i],'->',item)
                day_2_pvalcor[day_ls[i]] = item
        return(day_2_pval, day_2_pvalcor)
    
    @staticmethod
    def plot_ec50_surival_boxplot_OLD(day, df, outdir):
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        df = df.loc[df['Day'] == day]
        df = df.dropna(how='any')
        df['Ec50'] = -np.log10(df['Ec50'])
        cat_set = set(df['Category'].tolist())
        print(cat_set)
        if len(cat_set) != 2:
            return None

        #man-whitney test
        survival_ar = df.loc[df['Category'] == 'Survival']['Ec50'].tolist()
#         print(survival_ar)
        nonSurvival_ar = df.loc[df['Category'] == 'Non-survival']['Ec50'].tolist()
#         print(nonSurvival_ar)
        ustat,pval = stats.mannwhitneyu(survival_ar, nonSurvival_ar, alternative='two-sided')
#         print('Pval',pval)
        #
        sns.set(style="ticks")
        f, ax = plt.subplots(figsize=(7, 6))
        sns.boxplot(x="Category", y="Ec50",data=df, palette="vlag", order=["Survival","Non-survival"])
        sns.swarmplot(x="Category", y="Ec50",data=df, color=".3", linewidth=0, size=5,\
                      order=["Survival","Non-survival"])
        add_stat_annotation(ax, data=df, x="Category", y="Ec50", box_pairs=[("Survival","Non-survival")],\
                           test='Mann-Whitney', comparisons_correction=None, text_format='star',\
                            loc='outside', verbose=2)
        ax.xaxis.grid(False)
        ax.yaxis.grid(True)
        ax.set(ylabel="-log10(EC50)")
        ax.text(0.01,0.1, 'Pval: {:.2e}'.format(pval), transform=ax.transAxes, fontsize=5, verticalalignment='bottom')
        sns.despine(trim=True, left=False)
        outfile = outdir + '/Survival_Vs_Nonsurvival_day_' + str(day) + '.pdf'
        plt.title("Day after onset: " + str(day), pad=40)
        plt.savefig(outfile, dpi=300, bbox_inches='tight')
        f.clear()
        plt.close(f)

    @staticmethod
    def plot_ec50_surival_boxplot(day, df, outdir, day_2_pval, day_2_pvalcor, min_size, convert2Log,\
                                  targetVar, ymin, ymax, cat_order, palette_order):
#         print('CONIO')
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        df = df.loc[df['Day'] == day]
        df = df.dropna(how='any')
        if convert2Log == True:
            df['Var'] = -np.log10(df['Var'])
        cat_set = set(df['Category'].tolist())
        print(cat_set)
#         if len(cat_set) != 2:
#             return None

        sns.set(style="ticks")
        f, ax = plt.subplots(figsize=(7, 6))
        sns.boxplot(x="Category", y="Var",data=df, order=cat_order, palette=palette_order)
        sns.swarmplot(x="Category", y="Var",data=df, color=".3", linewidth=0, size=5,\
                          order=cat_order)
#         if len(cat_set) == 2:
#             add_stat_annotation(ax, data=df, x="Category", y="Var", box_pairs=[("Survival","Non-survival")],\
#                                test='Mann-Whitney', comparisons_correction=None, text_format='star',\
#                                 loc='outside', verbose=2)
#             if day in day_2_pval.keys():
#                 pval = day_2_pval[day]
#                 pvalcor = day_2_pvalcor[day]
# #                 print('Plot\t{}\t{}\t{}'.format(day, pval, pvalcor))
#                 ax.text(0.01,1.0, 'Day:{}\nPval: {:.3e}\ncorrectedPval: {:.3e}'.format(day, pval, pvalcor), transform=ax.transAxes, fontsize=5, verticalalignment='bottom')
        
        ax.xaxis.grid(False)
        ax.yaxis.grid(True)
        ax.set(ylabel=targetVar)
#         sns.despine(trim=True, left=False)
        outfile = outdir + '/Survival_Vs_Nonsurvival_day_' + str(day) + '.pdf'
        plt.title("Day after onset: " + str(day), pad=40)
#         print(ymax)
        if ymax > 0:
            plt.ylim(ymin,ymax)
#             print('joder')
        plt.savefig(outfile, dpi=300, bbox_inches='tight')
        f.clear()
        plt.close(f)    
    
    @staticmethod
    def plot_targetVar_violinplot(day, df, outdir, convert2Log, targetVar, ymin, ymax, cat_order, palette_order):
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        if day != 'All':
            df = df.loc[df['Day'] == day]
        else:
            df = df.loc[df['Day'] < 20]
        df = df.dropna(how='any')
        if convert2Log == True:
            df['Var'] = -np.log10(df['Var'])
        cat_set = set(df['Category'].tolist())
        
        sns.set(style="ticks")
        f, ax = plt.subplots(figsize=(7, 6))

        sns.violinplot(x="Day", y="Var",data=df, hue='Category',split = True, \
                       scale='count', inner='stick', scale_hue=False, bw=2, hue_order=cat_order, palette=palette_order)
        
        ax.xaxis.grid(False)
        ax.yaxis.grid(True)
        ax.set(ylabel=targetVar)
        sns.despine(trim=True, left=False)
        outfile = outdir + '/Survival_Vs_Nonsurvival_day_' + str(day) + '.pdf'
        plt.title("Day after onset: " + str(day), pad=40)
        if ymax > 0:
            plt.ylim(ymin,ymax)
        plt.savefig(outfile, dpi=300, bbox_inches='tight')
        f.clear()
        plt.close(f)          

    @staticmethod
    def plot_ec50_category_boxplot(day, df, outdir, cat_type, cat_order, ymin, ymax, targetVar, palette_order):
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        df = df.loc[df['Day'] == day]

        f, ax = plt.subplots(figsize=(7, 6))
#         sns.boxplot(x="Category", y="Var",data=df, palette=['lightgreen','navajowhite','lightblue', 'indianred'], order=cat_order)
        sns.boxplot(x="Category", y="Var",data=df, hue_order=cat_order, palette = palette_order, order = cat_order)
        sns.swarmplot(x="Category", y="Var",data=df, color=".3", linewidth=0, size=5, hue_order=cat_order, order = cat_order)

        ax.xaxis.grid(False)
        ax.yaxis.grid(True)
        ax.set(ylabel="-log10(EC50)")
#         ax.text(0.01,0.1, 'Pval: {:.2e}'.format(pval), transform=ax.transAxes, fontsize=5, verticalalignment='bottom')
#         sns.despine(trim=True, left=False)
        outfile = outdir + '/' + targetVar + '_' + cat_type + '_day_' + str(day) + '.pdf'
        plt.title("Day after onset: " + str(day), pad=40)
        if ymax > 0:
            plt.ylim(ymin,ymax)
        plt.ylabel(targetVar)
        plt.savefig(outfile, dpi=300, bbox_inches='tight')
        f.clear()
        plt.close(f)        
        
    @staticmethod
    def plot_ec50_2category_boxplot(day, df, outdir, cat_type, cat_order, ymin, ymax):
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        df = df.loc[df['Day'] == day]

        #man-whitney test
        cat1_ar = df.loc[df['Category'] == cat_order[0]]['-log10(EC50)'].tolist()
        
        cat2_ar = df.loc[df['Category'] == cat_order[1]]['-log10(EC50)'].tolist()
        
        ustat,pval = stats.mannwhitneyu(cat1_ar, cat2_ar, alternative='two-sided')        
        
        f, ax = plt.subplots(figsize=(7, 6))
        sns.boxplot(x="Category", y="-log10(EC50)",data=df, palette="vlag", order=cat_order)
        sns.swarmplot(x="Category", y="-log10(EC50)",data=df, color=".3", linewidth=0, size=5,
                      order=cat_order)
        if len(cat1_ar) > 2 and len(cat2_ar) > 2:
            add_stat_annotation(ax, data=df, x="Category", y="-log10(EC50)", box_pairs=[(cat_order[0],cat_order[1])],
                            test='Mann-Whitney', comparisons_correction=None, text_format='star',
                            loc='outside', verbose=2)
        
        ax.xaxis.grid(False)
        ax.yaxis.grid(True)
        ax.set(ylabel="-log10(EC50)")
        ax.text(0.01,0.1, 'Pval: {:.2e}'.format(pval), transform=ax.transAxes, fontsize=5, verticalalignment='bottom')
#         sns.despine(trim=True, left=False)
        outfile = outdir + '/' + cat_type + '_day_' + str(day) + '.pdf'
        plt.title("Day after onset: " + str(day), pad=40)
        if ymax > 0:
            plt.ylim(ymin,ymax)
        plt.savefig(outfile, dpi=300, bbox_inches='tight')
        f.clear()
        plt.close(f)             
        
    @staticmethod
    def plot_ec50_survival_trajectory(df, outfile, convert2Log, targetVar, plotstyle, estimat, x_lim, y_lim, cat_order, palette_order):
        df8 = pd.concat([df.loc[df['Category'] == 'Survival'],df.loc[df['Category'] == 'Non-survival']])
        df8 = df8.dropna(how='any')
        if convert2Log == True:
            df8['Ec50'] = -np.log10(df8['Var'])
            #df8.rename(columns={'Ec50': '-log10(EC50)'}, inplace=True)

#         fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,10))
#         sns.boxplot(x='Day', y='-log10(EC50)', hue='Category', data=df8.loc[df8['Category'] == 'Survival'], ax = ax1, color='blue')
#         sns.boxplot(x='Day', y='-log10(EC50)', hue='Category', data=df8.loc[df8['Category'] == 'Non-survival'], ax = ax2, color='red')
#         plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
#         plt.setp(ax2.xaxis.get_majorticklabels(), rotation=90)
#         plt.xlabel('Day after onset')
#         ax1.set_ylim(0, 10)
#         ax2.set_ylim(0, 10)
#         plt.savefig('08_2020/bloxplot_trajectory_sidebyside.pdf', dpi=300)
#         fig.clear()
#         plt.close(fig)

        fig, ax = plt.subplots(figsize=(20,6))
        # sns.palplot(sns.color_palette("RdBu", n_colors=2))
        if plotstyle == 'boxplot':
            sns.boxplot(x='Day', y='Var', data=df8, hue='Category', ax = ax, hue_order=cat_order, palette=palette_order, order=cat_order)
        elif plotstyle == 'lineplot':
            if estimat != 'median':
                sns.lineplot(x='Day', y='Var', data=df8, hue='Category',err_style="band",ci=90, hue_order=cat_order, palette=palette_order)
            else:
                sns.lineplot(x='Day', y='Var', data=df8, hue='Category',err_style="band",ci=90, \
                            estimator=np.median, hue_order=cat_order, palette=palette_order)
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
        plt.xlabel('Day after onset')
        plt.ylabel(targetVar)
        if x_lim > 0:
            plt.xlim(0, x_lim)
        if y_lim > 0:
            plt.ylim(0,y_lim)
        plt.savefig(outfile, dpi=300)
        fig.clear()
        plt.close(fig)

    @staticmethod
    def boxplot_ec50_category_trajectory(df, outfile, x_lim, y_lim, targetVar, cat_order, palette_order):
        fig=plt.figure(figsize=(18, 10))
        sns.boxplot(x='Day', y='Var', data=df, hue='Category', hue_order=cat_order, palette=palette_order, order=cat_order)
#         plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
        plt.xlabel('Day after onset')
        plt.ylabel(targetVar)
        plt.xticks(rotation=90)
        if x_lim != '':
            plt.xlim(0, x_lim)
        if y_lim != '':
            plt.ylim(0,y_lim)
        plt.savefig(outfile, dpi=300)   
        fig.clear()
        plt.close(fig)
        
    @staticmethod
    def lineplot_ec50_category_trajectory(df, outfile,x_lim, y_lim, targetVar, estimat, cat_order, palette_order, x_min, y_min):
        fig=plt.figure(figsize=(18, 10))
        if estimat == 'median':
            sns.lineplot(x='Day', y='Var', data=df, hue='Category',err_style="band",ci=90, \
                         estimator= np.median, hue_order = cat_order, palette=palette_order)
        else:
            sns.lineplot(x='Day', y='Var', data=df, hue='Category',err_style="band",ci=90, \
                         estimator= estimat, hue_order = cat_order, palette=palette_order)            
#         plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
        plt.xlabel('Day after onset')
        plt.ylabel(targetVar)
        plt.xticks(rotation=90)
        if x_lim != '':
            plt.xlim(x_min, x_lim)
        if y_lim != '':
            plt.ylim(y_min,y_lim)
        plt.savefig(outfile, dpi=300)   
        fig.clear()
        plt.close(fig)
        
    @staticmethod
    def logtransform_df_ec50_cat(df):
        df.dropna(how='any', inplace=True)
        df['-log10(EC50)'] = -np.log10(df['Ec50'])
        df.drop(columns=['Ec50'], inplace=True)
        return(df)
    
    @staticmethod
    def average_EC50_slidingWindow(df, dif):
        col_last = df.columns[len(df.columns)-1]
        df6 = df.copy()
        for patient, row_ser in df.iterrows():
        #     print(patient)
        #     print(row_ser)
            for col_head, val0 in row_ser.iteritems():
        #         print('\t', col_head, val)
                lower_col = col_head - dif
                if int(lower_col) < 0:
                    lower_col = 0
                upper_col = col_head + dif
                if upper_col > col_last:
                    upper_col = col_last
                val_ar = np.array([])
                average = np.nan
                for col in range (lower_col, upper_col+1):
                    if not col in row_ser.index.tolist():
                        continue
        #             print('column head is ', col)
                    val1 = row_ser.loc[col]
#                     print('\t\t',patient, col, val1)
                    if np.isnan(val1):
                        continue
                    val_ar = np.append(val_ar, val1)
        #         print(val_ar)
                if len(val_ar) > 0:
                    average = np.average(val_ar)
                df6.at[patient,col_head] = average
        return(df6)     

    @staticmethod
    def all_against_all_pairwise_mann_whitney(df_all, day, outdir, outfile, cat_order):
#         if not os.path.isdir(outdir):
#             os.mkdir(outdir)
        df = df_all.loc[df_all['Day'] == day].copy()
#         display(df)
        outfile = outdir + outfile
        if day == 0 and os.path.exists(outfile):
            os.remove(outfile)
            f = open(outfile,'w')
        elif day == 0:  
            f = open(outfile,'w')
        elif os.path.exists(outfile):
            f = open(outfile,'a')
            
        print('Day:\t' + str(day), file=f)
#         df = df.loc[df['Day'] == day]
        val_dict = {}
        for cat in cat_order:
            ar = np.array(df[df['Category'] == cat]['Var'].tolist())
#             print('joder', ar)
            val_dict[cat] = ar
        for i, cat1 in enumerate(val_dict):
            ar1 = val_dict[cat1]
            ar1 = ar1[np.logical_not(np.isnan(ar1))]
#             print(ar1)
 #         print(cat1)
            for j in range(i+1,len(val_dict)):
                cat_ar = [*val_dict]
                cat2 = cat_ar[j]
                ar2 = val_dict[cat2]
                ar2 = ar2[np.logical_not(np.isnan(ar2))]
#             print(ar1)
#             print(ar2)
                if len(ar1) >= 3 and len(ar2) >= 3:
#                     print(day)
#                     print(ar1)
#                     print(ar2)
#                     print()
                    try:
                        ustat,pval = stats.mannwhitneyu(ar1, ar2, alternative='two-sided') 
                    except:
                        pval = np.nan
                else:
                    pval = np.nan
                print('\t'+ cat1 + '\tVs\t' + cat2 + '\t->\tpval: ' + str(pval), file=f)
                print('\t\t' + cat1 + ': ' + str(len(ar1)) + ' instances', file=f)
                print('\t\t' + cat2 + ': ' + str(len(ar2)) + ' instances', file=f)
        print('###################################', file=f)   
        f.close()